In [35]:
import os, re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [36]:
h_map = {2: 0.0375, 3: 0.084, 6: 0.1575}
flux_map = {88: 25900, 78: 21250, 73: 19400}
abs_map = {0: 3, 92: 100}
surf_map = {0: 0.98, 1: 0.76}
pattern = r"h(\d+)_flux(\d+)_abs(\d+).*?_surf(\d+).*?_(\d+)s"
# Depending on where u store the data files
# cd drive/MyDrive/Colab\ Notebooks/Theoretical_VTDP
DATA_DIR = "../../data/Theoretical_VTDP"
DROP_COLS = ["TC_9_5", "TC_Bottom_rec_groove", "TC_wall_ins_ext", "TC_bottom_ins_groove", "Theoretical_Temps_11"]

In [37]:
def parse_filename_params(filename):
    match = re.search(pattern, filename)
    if not match:
        return None
    h = h_map.get(int(match[1]))
    flux = flux_map.get(int(match[2]))
    abs_val = abs_map.get(int(match[3]))
    surf = surf_map.get(int(match[4]))
    min_time = int(match[5])
    return h, flux, abs_val, surf, min_time

def load_and_process_file(path, h, flux, abs_val, surf, min_time):
    df = pd.read_csv(path, encoding="utf-8-sig")
    df = df[df["Time"] >= min_time].copy()
    df.drop(columns=[col for col in df.columns if col in DROP_COLS or col.startswith("Depth_")], inplace=True)
    df["h"] = h
    df["flux"] = flux
    df["abs"] = abs_val
    df["surf"] = surf
    return df

In [38]:
dataframes = []
for fname in os.listdir(DATA_DIR):
    if not fname.endswith(".csv"):
        continue
    params = parse_filename_params(fname)
    if params is None or None in params:
        print(f"Skipping: {fname}")
        continue
    path = os.path.join(DATA_DIR, fname)
    df = load_and_process_file(path, *params)
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

Skipping: h2_flux88_abs25_newSalt_surf0_172s - Sheet1_processed.csv
Skipping: h2_flux88_abs25_newSalt_wr_surf0_123s - Sheet1_processed.csv
Skipping: h2_flux88_abs25_surf0_493s - Sheet1_processed.csv
Skipping: h2_flux88_abs25_wr_surf0_393s - Sheet1_processed.csv
Skipping: h2_flux88_abs25_wr_surfParAdded_169s - Sheet1_processed.csv
Skipping: h2_flux88_abs25_wr_surfSimD_525s - Sheet1_processed.csv
Skipping: h3_flux88_abs25_mr_surf0_796s-Sheet1_processed.csv
Skipping: h3_flux88_abs25_surf0_439s-Sheet2_processed.csv
Skipping: h3_flux88_abs25_surf0_660s-Sheet1_processed.csv
Skipping: h3_flux88_abs25_wr_surf0_422s-Sheet1_processed.csv
Skipping: h3_flux88_abs25_wr_surf0_746s-Sheet2_processed.csv
Skipping: h3_flux88_abs90_mr_surf0_506s-Sheet1_processed.csv
Skipping: h3_flux88_abs90_surf0_692s-Sheet2_processed.csv
Skipping: h3_flux88_abs90_surf0_747s-Sheet3_processed.csv
Skipping: h3_flux88_abs90_surf0_749s-Sheet1_processed.csv
Skipping: h3_flux88_abs90_surf0_Redone_640s-Sheet3_processed.csv
Ski

In [39]:
theory_cols = [c for c in data.columns if c.startswith("Theoretical_Temps_")]
X = data[["Time", "h", "flux", "abs", "surf"] + theory_cols]
y = data.drop(columns=["Time", "h", "flux", "abs", "surf"] + theory_cols)

X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=1)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [40]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return self.fc4(x)

input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = NeuralNet(input_size, output_size)

In [41]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 500

for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.6f}")

Epoch [50/500], Loss: 0.041132
Epoch [100/500], Loss: 0.021496
Epoch [150/500], Loss: 0.015513
Epoch [200/500], Loss: 0.014513
Epoch [250/500], Loss: 0.013608
Epoch [300/500], Loss: 0.013070
Epoch [350/500], Loss: 0.012740
Epoch [400/500], Loss: 0.012551
Epoch [450/500], Loss: 0.012442
Epoch [500/500], Loss: 0.012352


In [42]:
model.eval()
with torch.no_grad():
    preds = model(X_test).numpy()

preds_real = y_scaler.inverse_transform(preds)
y_real = y_scaler.inverse_transform(y_test.numpy())
rmse = np.sqrt(np.mean((preds_real - y_real) ** 2, axis=0))

print("\nRMSE per output (°C):")
for col, val in zip(y.columns, rmse):
    print(f"{col}: {val:.3f} °C")

print(f"\nAverage RMSE across all outputs: {np.mean(rmse):.3f} °C")

torch.save(model.state_dict(), "thermal_model_weights.pth")
print("Model weights saved to 'thermal_model_weights.pth'")


RMSE per output (°C):
TC1_tip: 41.517 °C
TC2: 41.169 °C
TC3: 41.581 °C
TC4: 40.554 °C
TC5: 40.423 °C
TC6: 40.015 °C
TC7: 40.632 °C
TC8: 41.255 °C
TC9: 42.642 °C
TC10: 44.001 °C

Average RMSE across all outputs: 41.379 °C
Model weights saved to 'thermal_model_weights.pth'


In [43]:
new_input = [[1049.0, 0.1575, 25900, 3, 0.98] + [350.0]*10]  # Replace with actual physics model output
new_input_scaled = X_scaler.transform(new_input)
new_tensor = torch.tensor(new_input_scaled, dtype=torch.float32)

with torch.no_grad():
    pred = model(new_tensor).numpy()
real_pred = y_scaler.inverse_transform(pred)
print("\nPredicted Temperatures:", real_pred)



Predicted Temperatures: [[343.844   342.78424 345.74146 348.67227 352.48578 352.0921  343.4293
  344.8404  343.5476  329.33377]]


C:\Users\Rajasekar\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
# so for this model, the training data will be
# h (depth), flux (q0), abs (absorption coefficient k), surf (surface emisisivty epsilon), and varying time (tStep)
# and y will be the thermal couples values FOR THAT TIME
data = pd.read_csv("PG_combined_data.csv")

# removing these two columns, some data files have these, some don't. I'll worry about them later +
# need to ask what they do again, idk why there is a 11th temp
data.drop(columns=["TC_9_5", "TC_Bottom_rec_groove","TC_wall_ins_ext", "TC_bottom_ins_groove", "Theoretical_Temps_11"], axis=1, inplace=True)
depth_cols = [c for c in data.columns if c.startswith("Depth_")]


data.drop(columns=depth_cols, inplace=True)

# making sure no nulls in  other  rows after r emoving these two
print("Null Check: ", data.isnull().sum())

Theoretical_temps = [x for x in data.columns if x.startswith("Theoretical_Temps_")]
X = data[["Time", "h", "flux", "abs", "surf"] + Theoretical_temps]
print("INPUTS: \n", X.head())
y = data.drop(columns=["Time", "h", "flux", "abs", "surf"] + Theoretical_temps)
print("OUPUTS: \n", y.head())

y_columns = y.columns
# MinMaxScaler -> normalization, standardize when data is not normal (not in our case)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)




input_size = X_train.shape[1]
print("input size: ", input_size)

output_size = y_train.shape[1]
print("output size: ", output_size)

print("Amount of Training Data: ", X_train.shape[0])






FileNotFoundError: [Errno 2] No such file or directory: 'PG_combined_data.csv'

In [48]:
# basic neural net
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = NeuralNet(input_size, output_size)

# coommon loss func n optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 500
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad() # resetting gradients
    outputs = model(X_train)

    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}")

# no more calculating grad descent/back propgation, only forward
model.eval()
with torch.no_grad():
    y_pred = model(X_test).numpy()

# change back predictions and true values
y_pred_real = y_scaler.inverse_transform(y_pred)
y_test_real = y_scaler.inverse_transform(y_test.numpy())

# get RMSE per output
rmse_per_output = np.sqrt(np.mean((y_pred_real - y_test_real)**2, axis=0))
rmse_overall = np.mean(rmse_per_output)


print("\nRMSE per output (°C):")
for col, rmse in zip(y.columns, rmse_per_output):
    print(f"{col}: {rmse:.3f} °C")

print(f"\nAverage RMSE across all outputs: {rmse_overall:.3f} °C")

torch.save(model.state_dict(), "thermal_model_weights.pth")
print("Model weights saved to 'thermal_model_weights.pth'")

Epoch [50/500], Loss: 0.04558698460459709
Epoch [100/500], Loss: 0.02143028937280178
Epoch [150/500], Loss: 0.016544368118047714
Epoch [200/500], Loss: 0.014925160445272923
Epoch [250/500], Loss: 0.014348020777106285
Epoch [300/500], Loss: 0.013880021870136261
Epoch [350/500], Loss: 0.013423283584415913
Epoch [400/500], Loss: 0.013083991594612598
Epoch [450/500], Loss: 0.012842696160078049
Epoch [500/500], Loss: 0.012653527781367302

RMSE per output (°C):
TC1_tip: 41.745 °C
TC2: 41.108 °C
TC3: 41.890 °C
TC4: 42.209 °C
TC5: 42.173 °C
TC6: 40.877 °C
TC7: 40.865 °C
TC8: 41.311 °C
TC9: 42.706 °C
TC10: 44.118 °C

Average RMSE across all outputs: 41.900 °C
Model weights saved to 'thermal_model_weights.pth'
